In [99]:
def parse(data):
    return np.array([[0 if x == '.' else 1 for x in line] for line in data.strip().split('\n')])

In [8]:
seating = parse(data)


In [10]:
from scipy.signal import convolve2d

In [88]:
def print_seating(seating, occupied):
    vals = seating + 2*occupied
    s = [[ ('.','L','?','#')[i] for i in line ] for line in vals]
    s = "\n".join(["".join(line) for line in s])
    print(s)
    print()

In [151]:
def simulate(seating):
    occupied = np.zeros_like(seating)
    prev_hash = None
    while True:
        rule1 = np.array([[1,1,1],[1,1,1],[1,1,1]])
        rule2 = np.array([[1,1,1],[1,10,1],[1,1,1]])
        filled = convolve2d(rule1, occupied)[1:-1,1:-1] == 0
        emptied = convolve2d(rule2, occupied)[1:-1,1:-1] >= 14
        occupied[filled] = 1
        occupied[emptied] = 0
        occupied *= seating
        h = hash(occupied.tobytes())
        if h == prev_hash:
            return np.sum(occupied)
        prev_hash = h


In [152]:
data = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""
seating = parse(data)
simulate(seating)

37

In [153]:
with open('11.txt', 'r') as f:
    seating = parse(f.read())
simulate(seating)

2277

In [146]:
from collections import defaultdict
def find_neighbors(seating):
#     neighbors = [[[] for _ in row] for row in seating]
    neighbors = defaultdict(list)

    N, M = seating.shape
    for i in range(N):
        for j in range(M):
            if seating[i,j] == 1:
                for di, dj in [(1,0),(0,1),(1,1),(1,-1)]:
                    for k in range(1,min(N,M)):
                        ii, jj = i+k*di, j+k*dj
#                         print((i,j),(ii,jj))
                        if not ( 0 <= ii < N and 0 <= jj < M):
                            break
                        if seating[ii,jj] == 1:
                            neighbors[(i,j)].append((ii,jj))
                            neighbors[(ii,jj)].append((i,j))
                            break
    return neighbors

In [168]:
data = """L.LL.LL.LL
LLLLLLL.LL
L.L.L..L..
LLLL.LL.LL
L.LL.LL.LL
L.LLLLL.LL
..L.L.....
LLLLLLLLLL
L.LLLLLL.L
L.LLLLL.LL
"""


In [174]:
def simulate(seating):
    occupied = np.zeros_like(seating)
    neighbors = find_neighbors(seating)
    prev_hash = None
    N, M = occupied.shape
    while True:
        new_occupied = occupied.copy()
        for (i,j), ns in neighbors.items():
            
            if occupied[i,j] == 0:
                if all(occupied[ii,jj] == 0 for ii, jj in ns):
                    new_occupied[i,j] = 1
            else:
                if sum(occupied[ii,jj] for ii, jj in ns) >= 5:
                    new_occupied[i,j] = 0
        occupied = new_occupied
#         print_seating(seating, occupied)
        h = hash(occupied.tobytes())
        if h == prev_hash:
            return occupied.sum()
        prev_hash = h

In [175]:
seating = parse(data)
simulate(seating)

26

In [176]:
with open('11.txt', 'r') as f:
    seating = parse(f.read())
simulate(seating)

2066